In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-vjovk4c4
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-vjovk4c4
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=6e9925745ed74b31b7f5cf3dca4371850a80f7215a7213ef925e2c9760f84023
  Stored in directory: /tmp/pip-ephem-wheel-cache-xsmu03k9/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [16]:
%%cu
#include <stdio.h>
#define DIM 1000

struct cuComplex{
    float r;
    float i;
    cuComplex(float a, float b):r(a), i(b){}
    float magnitude2(void){return r*r+i*i;}
    cuComplex operator*(const cuComplex &a) {
        return cuComplex(r*a.r-i*a.i,i*a.r+r*a.i);
    }
    cuComplex operator+(const cuComplex &a){
        return cuComplex(r+a.r,i+a.i);
    }
};

int julia(int x, int y){
    const float scale = 1.5;
    float jx = scale * (float)(DIM/2-x) / (DIM/2);
    float jy = scale * (float)(DIM/2-y) / (DIM/2);
    cuComplex c(-0.8, 0.156);
    cuComplex a(jx, jy);
    int i = 0;
    for(i = 0; i < 200; i++){
        a=a*a+c;
        if(a.magnitude2()>1000) return 0;
    }
    return 1;
}

void kernel(unsigned char *ptr){
    for(int y=0; y<DIM; y++){
        for(int x=0; x<DIM; x++){
            int offset = x+y*DIM;
            int juliaValue = julia(x,y);
            ptr[offset*4+0] = 255 * juliaValue;
            ptr[offset*4+1] = 0;
            ptr[offset*4+2] = 0;
        }
    }
}

//Zapis fraktala
struct DataBlock{
    unsigned char *dev_bitmap;
};

void wypisz_RGB_fraktal(unsigned char *ptr){
    for(int y=0; y<DIM; y++){
        for(int x=0; x<DIM; x++){
            int offset = x+y*DIM;

            printf("\n%d, %d, %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
        }
    }
}

void save_to_file(unsigned char *ptr){
    FILE *fp=fopen("Fraktal_CPU.txt","w");
    for(int y=0; y<DIM; y++){
        for(int x=0; x<DIM; x++){
            int offset=x+y*DIM;
            fprintf(fp, "\n%d, %d, %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
        }
    }
    fclose(fp);
}

int main(void){
    DataBlock data;
    unsigned char *bitmap = (unsigned char*)malloc(DIM*DIM*4*sizeof(unsigned char));
    int image_size = DIM*DIM*4;
    kernel(bitmap);
    save_to_file(bitmap);
}

In [8]:
%%cu
#include <stdio.h>
#define DIM 1000

struct cuComplex {
float r;
float i;
//cuComplex( float a, float b ) : r(a), i(b) {}
__device__ cuComplex(float a, float b):r(a), i(b){}
__device__ float magnitude2( void ) {
return r * r + i * i;
}
__device__ cuComplex operator*(const cuComplex &a) {
return cuComplex(r*a.r - i*a.i, i*a.r + r*a.i);
}
__device__ cuComplex operator+(const cuComplex &a) {
return cuComplex(r+a.r, i+a.i);
}
};

__device__ int julia( int x, int y ) {
const float scale = 1.5;
float jx = scale * (float)(DIM/2 - x)/(DIM/2);
float jy = scale * (float)(DIM/2 - y)/ (DIM/2);
cuComplex c(-0.8, 0.156);
cuComplex a(jx, jy);
int i = 0;
for (i =0; i <200; i++) {
a = a * a + c;
if (a.magnitude2() > 1000)
return 0;
}
return 1;
}

__global__ void kernel( unsigned char *ptr ) {
// Odwzorowanie z blockldx na współrzędne piksela
int x = blockIdx.x;
int y = blockIdx.y;
int offset = x + y * gridDim.x;
// Obliczenie wartości dla tego punktu
int juliaValue = julia ( x, y );
ptr[offset*4 + 0] = 255 * juliaValue;
ptr[offset*4 + 1] = 0;
ptr[offset*4 +2] =0;
ptr[offset*4 + 3] = 255;
}

//Zapis fraktala
struct DataBlock{
    unsigned char *dev_bitmap;
};

void wypisz_RGB_fraktal(unsigned char *ptr){
    for(int y=0; y<DIM; y++){
        for(int x=0; x<DIM; x++){
            int offset = x+y*DIM;

            printf("\n%d, %d, %d, %d", ptr[offset*4+0], ptr[offset*4+1],
                   ptr[offset*4+2], ptr[offset*4+3]);
        }
    }
}

void save_to_file(unsigned char *ptr){
    FILE *fp=fopen("Fraktal_GPU.txt","w");
    for(int y=0; y<DIM; y++){
        for(int x=0; x<DIM; x++){
            int offset=x+y*DIM;
            fprintf(fp, "\n%d, %d, %d, %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2], ptr[offset*4+3]);
        }
    }
    fclose(fp);
}

int main(void){
DataBlock data;
unsigned char *bitmap = (unsigned char*)malloc(DIM*DIM*4*sizeof(unsigned char));
int image_size = DIM*DIM*4;
unsigned char *dev_bitmap;
cudaMalloc((void**)&dev_bitmap, image_size);
data.dev_bitmap=dev_bitmap;
dim3 grid(DIM,DIM);
kernel<<<grid,1>>>(dev_bitmap);
cudaMemcpy(bitmap,dev_bitmap,image_size,cudaMemcpyDeviceToHost);
cudaFree(dev_bitmap);
save_to_file (bitmap);
}


In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
image = mpimg.imread("asd.png")
plt.imshow(image)
plt.show()

ValueError: ignored